In [ ]:
# A Practical Guide to Binary Classification

In [1]:
# Step 1: Import Libraries and Load the Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from time import time
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score
from matplotlib import pyplot
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier, PassiveAggressiveClassifier, Perceptron
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/fenago/datasets/main/bank-full.csv', sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
#Step 2: Preprocessing the Data
df.fillna(0, inplace=True)
print(df.nunique())

age            77
job            12
marital         3
education       4
default         2
balance      7168
housing         2
loan            2
contact         3
day            31
month          12
duration     1573
campaign       48
pdays         559
previous       41
poutcome        4
y               2
dtype: int64


In [4]:
# Step 3: Label Encoding the Target Variable
from sklearn.preprocessing import LabelEncoder

# Instantiate the LabelEncoder
le = LabelEncoder()

# Fit and transform the binary column
df['y'] = le.fit_transform(df['y'])

# Print the mapping
mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping)

{'no': 0, 'yes': 1}


In [5]:
# Step 4: Feature and Target Separation
X = df.drop(['y'], axis=1)
y = df['y']

In [6]:
# Step 5: One-Hot Encoding for Categorical Variables
X = pd.get_dummies(X)
X.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5,261,1,-1,0,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1,44,29,5,151,1,-1,0,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,33,2,5,76,1,-1,0,False,False,True,...,False,False,True,False,False,False,False,False,False,True
3,47,1506,5,92,1,-1,0,False,True,False,...,False,False,True,False,False,False,False,False,False,True
4,33,1,5,198,1,-1,0,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [7]:
# Step 6: Train-Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Step 7: Algorithm Harness for Model Comparison
models = []
models.append(('LR', LogisticRegression(solver='liblinear')))
models.append(('SGD', SGDClassifier()))
models.append(('Ridge', RidgeClassifier()))
models.append(('PAC', PassiveAggressiveClassifier()))
models.append(('Perceptron', Perceptron()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NearestCentroid', NearestCentroid()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('ExtraTree', ExtraTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('BNB', BernoulliNB()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('NuSVC', NuSVC()))
models.append(('LinearSVC', LinearSVC()))
models.append(('RF', RandomForestClassifier()))
models.append(('GB', GradientBoostingClassifier()))
models.append(('ExtraTrees', ExtraTreesClassifier()))
models.append(('Bagging', BaggingClassifier()))
models.append(('AdaBoost', AdaBoostClassifier()))
models.append(('MLP', MLPClassifier()))
models.append(('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss')))

In [ ]:
# Step 8: Model Evaluation and Cross-Validation
# Scoring metrics:
# 'accuracy', 'balanced_accuracy', 'average_precision', 'neg_brier_score', 'f1', 'f1_micro', 'f1_macro', 'f1_weighted',
# 'neg_log_loss', 'precision', 'recall', 'jaccard', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted'

results = []
names = []
scoring = 'accuracy'

for name, model in models:
    start = time()
    kfold = KFold(n_splits=10, random_state=7, shuffle=True)
    model.fit(X_train, y_train)
    train_time = time() - start
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    predict_time = time() - start
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    print("Score for each of the 10 K-fold tests: ", cv_results)
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print()

LR: 0.901709 (0.003658)
Score for each of the 10 K-fold tests:  [0.90489356 0.89687586 0.90295825 0.90572297 0.90710534 0.89881117
 0.8954935  0.90185236 0.90348451 0.89988938]
LogisticRegression(solver='liblinear')
	Training time: 2.635s
	Prediction time: 9.891s

SGD: 0.822024 (0.096493)
Score for each of the 10 K-fold tests:  [0.85540503 0.57063865 0.78490462 0.89024053 0.89245231 0.88443461
 0.87586398 0.84268731 0.7397677  0.88384956]
SGDClassifier()
	Training time: 0.752s
	Prediction time: 12.427s

Ridge: 0.900216 (0.004240)
Score for each of the 10 K-fold tests:  [0.90047    0.89494056 0.90212884 0.90489356 0.90572297 0.89825823
 0.89134642 0.90240531 0.90293142 0.89905973]
RidgeClassifier()
	Training time: 0.152s
	Prediction time: 1.677s

PAC: 0.776770 (0.166483)
Score for each of the 10 K-fold tests:  [0.49294996 0.8786287  0.88388167 0.890517   0.41719657 0.78739287
 0.87807575 0.77522809 0.87721239 0.88661504]
PassiveAggressiveClassifier()
	Training time: 0.362s
	Prediction t

/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/disc

QDA: 0.872788 (0.006786)
Score for each of the 10 K-fold tests:  [0.8656345  0.86342273 0.87973459 0.86978159 0.8855405  0.88111695
 0.86839923 0.87005806 0.87361726 0.87057522]
QuadraticDiscriminantAnalysis()
	Training time: 0.309s
	Prediction time: 2.393s

KNN: 0.882659 (0.004588)
Score for each of the 10 K-fold tests:  [0.88084048 0.87779928 0.89189936 0.88471109 0.88941111 0.87669339
 0.880564   0.88332873 0.88053097 0.88080752]
KNeighborsClassifier()
	Training time: 0.021s
	Prediction time: 29.289s

NearestCentroid: 0.736203 (0.005223)
Score for each of the 10 K-fold tests:  [0.73513962 0.73265137 0.74094553 0.73679845 0.7442632  0.73901023
 0.72822781 0.72850429 0.73423673 0.74225664]
NearestCentroid()
	Training time: 0.022s
	Prediction time: 0.462s

CART: 0.873369 (0.005516)
Score for each of the 10 K-fold tests:  [0.87337573 0.87005806 0.87586398 0.88001106 0.87696986 0.87724634
 0.86618745 0.87475809 0.87776549 0.86144912]
DecisionTreeClassifier()
	Training time: 0.523s
	Predi

In [ ]:
# Step 9: Visualization of Model Performance

fig = pyplot.figure(figsize=(15, 10))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names, rotation=45)
pyplot.show()